<a href="https://colab.research.google.com/github/NereaSchez/Clase2/blob/main/Practica_Banco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nueva sección

**Descargar BBDD**

In [ ]:
!gdown 1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX

Downloading...
From: https://drive.google.com/uc?id=1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX
To: /content/data_berka.db
100% 70.2M/70.2M [00:00<00:00, 173MB/s]


1.1) Cargamos la extensión sql (nos permite ejecutar comandos sql directamente en colab

In [ ]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [ ]:
%sql sqlite:///data_berka.db

1.2) Conectar a la base de datos

1.3) Comprobamos el esquema de la BBDD para ejecutar comandos sql, se debe añadir: "%sql

In [ ]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type = 'table'

   sqlite:///data-berka.db
 * sqlite:///data_berka.db
Done.


name,type
DISTRICT,table
ACCOUNT,table
TRANS,table
CLIENT,table
DISPOSITION,table
CREDIT_CARD,table
LOAN,table
PERMANENT_ORDER,table


# 2) Ejercicios

2.1) Muestre los datos de la tabla cliente

In [ ]:
%sql SELECT * FROM CLIENT LIMIT 5


   sqlite:///data-berka.db
 * sqlite:///data_berka.db
Done.


client_id,birth_number,district_id
1,706213,18
2,450204,1
3,406009,1
4,561201,5
5,605703,5


2.2) Liste todos los clientes con cuentas activas y préstamos, incluyendo dinero prestado y saldos de cuenta

In [ ]:
%sql SELECT * FROM ACCOUNT LIMIT 5

   sqlite:///data-berka.db
 * sqlite:///data_berka.db
Done.


account_id,district_id,frequency,date
576,55,monthly,930101
3818,74,monthly,930101
704,55,monthly,930101
2378,16,monthly,930101
2632,24,monthly,930102


In [ ]:
%sql SELECT * FROM LOAN LIMIT 5

   sqlite:///data-berka.db
 * sqlite:///data_berka.db
Done.


loan_id,account_id,date,amount,duration,payments,status
5314,1787,930705,96396,12,8033,B
5316,1801,930711,165960,36,4610,A
6863,9188,930728,127080,60,2118,A
5325,1843,930803,105804,36,2939,A
7240,11013,930906,274740,60,4579,A


In [ ]:
%sql SELECT * FROM disposition LIMIT 15

   sqlite:///data-berka.db
 * sqlite:///data_berka.db
Done.


disp_id,client_id,account_id,type
1,1,1,OWNER
2,2,2,OWNER
3,3,2,DISPONENT
4,4,3,OWNER
5,5,3,DISPONENT
6,6,4,OWNER
7,7,5,OWNER
8,8,6,OWNER
9,9,7,OWNER
10,10,8,OWNER


In [ ]:
%%sql SELECT
      c.client_id AS Cliente,
      ROUND(SUM(l.amount), 2) AS Importe_Prestado,
      ROUND(SUM(CASE
        WHEN t.type = 'PRIJEM' THEN t.amount
        WHEN t.type = 'VYDAJ' THEN -t.amount
        ELSE 0
        END), 2) AS Balance_Cuenta
FROM CLIENT c
LEFT JOIN DISPOSITION d ON d.client_id = c.client_id
LEFT JOIN ACCOUNT a On a.account_id = d.account_id
LEFT JOIN LOAN l ON l.account_id = a.account_id
LEFT JOIN TRANS t ON t.account_id = a.account_id
WHERE d.type = 'OWNER'
GROUP BY c.client_id
LIMIT 20

   sqlite:///data-berka.db
 * sqlite:///data_berka.db
Done.


Cliente,Importe_Prestado,Balance_Cuenta
1,None,13466.5
2,38695056.0,260069.7
4,None,51096.2
6,None,33720.6
7,None,28088.6
8,None,47667.9
9,None,68495.9
10,None,72617.3
12,None,23877.7
13,None,59295.5


2.3) Identifique clientes con múltiples cuentas.

In [ ]:
%%sql SELECT
      c.cliente_id AS Cliente,
      COUNT(a.account_id) AS Total


   sqlite:///data-berka.db
 * sqlite:///data_berka.db
(sqlite3.OperationalError) no such column: c.cliente_id
[SQL: SELECT
      c.cliente_id AS Cliente,
      COUNT(a.account_id) AS Total]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


2.4) Recupere transacciones del año 1998, ordenadas por cantidad de transacción, e identifique las 10 transacciones más grandes

In [ ]:
%%sql SELECT trans_id, account_id, date, type, operation, amount
FROM TRANS
WHERE strftime('%Y', date) = '1998'
ORDER BY amount DESC
LIMIT 10;

   sqlite:///data-berka.db
 * sqlite:///data_berka.db
Done.


trans_id,account_id,date,type,operation,amount


¿Cuál es el saldo promedio de cuentas con préstamos en comparación con aquellas sin préstamos?

SyntaxError: invalid syntax (ipython-input-71-270570092.py, line 1)

In [ ]:
# prompt: ¿Cuál es el saldo promedio de cuentas con préstamos en comparación con aquellas sin préstamos?

WITH AccountBalances AS (
    SELECT
        account_id,
        SUM(CASE
            WHEN type = 'PRIJEM' THEN amount
            WHEN type = 'VYDAJ' THEN -amount
            ELSE 0
        END) AS account_balance
    FROM TRANS
    GROUP BY account_id
),
AccountsWithLoans AS (
    SELECT DISTINCT account_id
    FROM LOAN
)
SELECT
    'Con Préstamos' AS TipoCuenta,
    AVG(ab.account_balance) AS SaldoPromedio
FROM AccountBalances ab
JOIN AccountsWithLoans awl ON ab.account_id = awl.account_id

UNION ALL

SELECT
    'Sin Préstamos' AS TipoCuenta,
    AVG(ab.account_balance) AS SaldoPromedio
FROM AccountBalances ab
LEFT JOIN AccountsWithLoans awl ON ab.account_id = awl.account_id
WHERE awl.account_id IS NULL;
